In [1]:
import pandas as pd
import numpy as np

In [4]:
def read_data() -> pd.DataFrame:
    b = pd.read_csv('baron-bounds.csv')
    b['LB_VALUE'] = b.obj

    c = pd.read_csv('baron-concave-bounds.csv')
    c['LB_VALUE'] = c.original_obj

    l = pd.read_csv('labelling-lower-bounds.csv')
    l['LB_VALUE'] = l.obj

    return pd.concat([b, c, l])

In [5]:
res = read_data()
bks = pd.read_csv('bks.csv')

pwl = pd.read_csv('model-based-upper-bounds.csv')
pwl = pwl[pwl.algorithm == 'integer_linear_model_LinearModelObjectiveFunction.LINEAR_APPROX_TIGHT_with_vi1_with_vi4_liftMTZ'].copy()

cnt = pd.read_csv('model-based-upper-bounds.csv')
cnt = cnt[cnt.algorithm == 'frankwolfe_with_vi1_with_vi4_liftMTZ'].copy()

In [6]:
def get_gap_with_bks(row: pd.Series) -> float:
    i = row.instance_basename
    bks_row = bks[bks.instance_basename == i]

    if len(bks_row) == 0:
        print(f"No entry found in BKS file for instance {i}")
        raise RuntimeError('Missing BKS value')

    ub = bks_row.iloc[0]['BKS']
    lb = row['LB_VALUE']

    if np.isnan(ub) or ub == 0:
        return 1
    else:
        return np.abs(ub - lb) / ub

def get_gap_with_cnt(row: pd.Series) -> float:
    i = row.instance_basename
    cnt_row = cnt[cnt.instance_basename == i]

    if len(cnt_row) == 0:
        print(f"No entry found in CNT file for instance {i}")
        raise RuntimeError('Missing CNT value')

    ub = cnt_row.iloc[0]['obj']
    lb = row['LB_VALUE']

    if np.isnan(ub) or ub == 0:
        return 1
    else:
        return np.abs(ub - lb) / ub

In [7]:
res['GAP_WITH_BKS'] = res.apply(get_gap_with_bks, axis=1)
res['GAP_WITH_CNT'] = res.apply(get_gap_with_cnt, axis=1)

In [9]:
res.groupby('algorithm').mean()[['GAP_WITH_BKS', 'GAP_WITH_CNT', 'time_s']].round(3)

,GAP_WITH_BKS,GAP_WITH_CNT,time_s
algorithm,,,
labelling_StrongLabel,0.472,0.574,3187.458
nonlinear_concave_model,0.002,0.161,647.858
nonlinear_concave_model_with_vi1_liftMTZ,0.000,0.159,70.467
nonlinear_model_with_vi1_liftMTZ,0.075,0.220,830.560


In [11]:
def print_vals(df: pd.DataFrame, last: bool = False) -> None:
    print(f"{100*df['GAP_WITH_BKS'].mean():>5.2f} & ", end='')
    print(f"{100*df['GAP_WITH_CNT'].mean():.2f} & ", end='')
    print(f"{df.time_s.mean():>6.1f}", end='')

    if last:
        print(' \\\\')
    else:
        print(' & ', end='')

In [15]:
print('\\begin{tabular}{ll rrr rrr rrr rrr}')
print('\\toprule')
print(' & & ', end='')
print('\\multicolumn{3}{c}{\\textsc{LB-NLModel}} & ', end='')
print('\\multicolumn{3}{c}{\\textsc{LB-NLCModel}} & ', end='')
print('\\multicolumn{3}{c}{\\textsc{LB-NLCModel}\\(^*\\)} & ', end='')
print('\\multicolumn{3}{c}{\\textsc{}LB-Label} \\\\')

print('\\cmidrule(lr){3-5}\\cmidrule(lr){6-8}\\cmidrule(lr){9-11}\\cmidrule(lr){12-14}')
print('\\(\\alpha^{\\text{Tsi}}\\) & \\(\\beta^{\\text{Tsi}}\\) & ', end='')
print('G\\textsubscript{1}\% & G\\textsubscript{2}\% & T (s) & ' * 3, end='')
print('G\\textsubscript{1}\% & G\\textsubscript{2}\% & T (s) \\\\')
print('\\midrule')

for alpha in sorted(res.tsiligirides_hop_alpha.unique()):
    for beta in sorted(res.tsiligirides_hop_beta.unique()):
        f = res[(res.tsiligirides_hop_alpha == alpha) & (res.tsiligirides_hop_beta == beta)]

        print(f"{alpha:.2f} & {beta} & ", end='')

        df = f[f.algorithm == 'nonlinear_model_with_vi1_liftMTZ']
        print_vals(df)

        df = f[f.algorithm == 'nonlinear_concave_model_with_vi1_liftMTZ']
        print_vals(df)

        df = f[f.algorithm == 'nonlinear_concave_model']
        print_vals(df)

        df = f[f.algorithm == 'labelling_StrongLabel']
        print_vals(df, last=True)

print('\\midrule')
print('\\multicolumn{2}{l}{Overall} & ', end='')

df = res[res.algorithm == 'nonlinear_model_with_vi1_liftMTZ']
print_vals(df)

df = res[res.algorithm == 'nonlinear_concave_model_with_vi1_liftMTZ']
print_vals(df)

df = res[res.algorithm == 'nonlinear_concave_model']
print_vals(df)

df = res[res.algorithm == 'labelling_StrongLabel']
print_vals(df, last=True)

print('\\bottomrule')
print('\\end{tabular}')

\begin{tabular}{ll rrr rrr rrr rrr}
\toprule
 & & \multicolumn{3}{c}{\textsc{LB-NLModel}} & \multicolumn{3}{c}{\textsc{LB-NLCModel}} & \multicolumn{3}{c}{\textsc{LB-NLCModel}\(^*\)} & \multicolumn{3}{c}{\textsc{}LB-Label} \\
\cmidrule(lr){3-5}\cmidrule(lr){6-8}\cmidrule(lr){9-11}\cmidrule(lr){12-14}
\(\alpha^{\text{Tsi}}\) & \(\beta^{\text{Tsi}}\) & G\textsubscript{1}\% & G\textsubscript{2}\% & T (s) & G\textsubscript{1}\% & G\textsubscript{2}\% & T (s) & G\textsubscript{1}\% & G\textsubscript{2}\% & T (s) & G\textsubscript{1}\% & G\textsubscript{2}\% & T (s) \\
\midrule
0.10 & 2 &  0.00 & 5.59 &   69.6 &  0.00 & 5.59 &   27.0 &  0.00 & 5.59 &  102.1 & 47.14 & 51.25 & 3177.7 \\
0.10 & 3 &  0.00 & 8.23 &  103.0 &  0.00 & 8.23 &   24.7 &  0.00 & 8.23 &  129.5 & 45.47 & 51.34 & 3188.4 \\
0.10 & 4 &  0.00 & 12.95 &  139.9 &  0.00 & 12.95 &   33.5 &  0.00 & 12.95 &  155.8 & 45.08 & 54.07 & 3182.6 \\
0.10 & 5 &  0.00 & 12.96 &  216.4 &  0.00 & 12.96 &   26.5 &  0.00 & 12.96 &  192.8 & 44.23 